In [13]:
import io
import os
import RPi.GPIO as GPIO
import time
import subprocess
import picamera
import numpy as np
from google.cloud import vision
from google.cloud.vision import types

TRIG = 11
ECHO = 12
MIN_V = 45
MAX_V = 80
MIN_T = 0.00011764705
MAX_T = 0.02352941176
TMP_FILE = 'resources/tmp.jpg'

# Part I: Regarding sound process
def sound_proc(freq):
    return subprocess.Popen(['speaker-test', '--frequency', str(freq), '--test', 'sine'])

def set_volume(percent):
    subprocess.call(['amixer', 'set', 'PCM', str.format('{}%', percent)])

# Part II: Regarding distance
def dur_to_vol(t):
    return MAX_V - ((MAX_V - MIN_V) / (MAX_T - MIN_T))*t

def duration():
    GPIO.output(TRIG, 0)
    time.sleep(0.000002)
    GPIO.output(TRIG, 1)
    time.sleep(0.00001)
    GPIO.output(TRIG, 0)
    while GPIO.input(ECHO) == 0: pass
    time1 = time.time()
    while GPIO.input(ECHO) == 1: pass
    time2 = time.time()
    t = time2 - time1
    if (MIN_T < t < MAX_T):
        return t

# Part III: Regarding color
def avg_color(rgb_array):
    return np.mean(rgb_array, axis=(0, 1)).astype(int)

def color_to_freq(c):
    r, g, b = (((c[k]) >> 4) for k in range(3))  # trim 4 bit from each value
    return 2 * ((r << 8) + (g << 4) + b)

def main():
    with picamera.PiCamera() as camera:
        camera.resolution = (320, 240)  # Scale down resolution
        try:
            # Google Vision setup
            client = vision.ImageAnnotatorClient()  # Instantiates a client
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "api-key.json"

            # Sonic setup
            GPIO.setmode(GPIO.BOARD)
            GPIO.setup(TRIG, GPIO.OUT)
            GPIO.setup(ECHO, GPIO.IN)

            # Color setup
            rgb_array = np.empty((240, 320, 3), dtype=np.uint8)

            # Turn on the sound
            pid = sound_proc(440)
            set_volume(MIN_V)

            counter = 0  # To keep count for Google Vision
            while True:
                counter += 1
                
                # Sonic part
                t = duration()  # Gather distance
                if t is not None:
                    v = dur_to_vol(t)
                    set_volume(v)

                # Color:
                camera.capture(rgb_array, 'rgb')
                freq = color_to_freq(avg_color(rgb_array))
                print(freq, end=' ')
                proc = sound_proc(freq)
                proc.kill()  # Kill the old frequency
                    
                # Vision part (not every cycle)
                if counter == 10:
                    counter = 0

                    # Capture the image to file...
                    camera.capture(TMP_FILE)
                    # ... just to load it back to memory
                    with io.open(TMP_FILE, 'rb') as image_file:
                        content = image_file.read()
                    jpg = types.Image(content=content)

                    # Process the image on the cloud
                    response = client.label_detection(image=jpg)
                    labels = response.label_annotations

                    print('Labels:')
                    for label in labels[:3]:
                        print(label.description)

                time.sleep(0.1)  # Periodic sleep to keep the process from being pre-empted

        except Exception as e:
            print(e)
        finally:
            GPIO.cleanup()
            pid.kill()

main()

3786 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3788 3276 3788 3786 4914 4366 5460 2696 3276 2150 2730 Labels:
ceiling
structure
daylighting
3276 3276 2184 2184 1604 2698 2184 2184 2184 

KeyboardInterrupt: 